In [1]:
import getpass
import os
from langchain_community.utilities import SQLDatabase

os.environ["OPENAI_API_KEY"] = "sk-proj-NZ8AFTmqmU9O18oruN6JT3BlbkFJ7qoug7ZXLhghtBlOI0QG"
db_config = {
    "user": "postgres",
    "password": "password",
    "host": "localhost", 
    "port": "5432", 
    "database": "hris" 
}


db = SQLDatabase.from_uri(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM employee LIMIT 10;")


postgresql
['absensi_harian', 'employee']


"[(26, 'Nanda', 'Purnama', 'caustickp@ucoz.ru', 'Male', 'internship', '11', 40, 'Sakura SSS', 5313000, 60), (30, 'Wahyu', 'Sari', 'bwisont@oracle.com', 'Female', 'business development', '19', 45, 'Sakura Internal', 6464000, 76), (37, 'Indra', 'Wijaya', 'cmcginley10@xrea.com', 'Male', 'operational', '5', 45, 'Sakura Krakatau Steel', 7510000, 96), (28, 'Putri', 'Kusuma', 'hortsmannr@archive.org', 'Female', 'junior manager', '5', 40, 'Sakura SSS', 7465000, 48), (29, 'Joko', 'Sari', 'mtodeos@go.com', 'Female', 'junior manager', '24', 30, 'Sakura Internal', 6127000, 47), (39, 'Oktavia', 'Sari', 'bshovell12@cdbaby.com', 'Male', 'operational', '4', 30, 'Sakura SSS', 5541000, 92), (31, 'Agus', 'Sari', 'drubinsohnu@princeton.edu', 'Male', 'part-time', '26', 20, 'Sakura Internal', 5308000, 40), (41, 'Wahyu', 'Sari', 'aguillet14@dot.gov', 'Female', 'part-time', '15', 20, 'Sakura Internal', 8060000, 51), (38, 'Putri', 'Sari', 'aspinozzi11@soup.io', 'Male', 'part-time', '11', 20, 'Sakura SSS', 4039

In [2]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [3]:
agent_executor.invoke(
    "List the total employees per company. Which company has most employees?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


absensi_harian, employee
Invoking: `sql_db_schema` with `{'table_names': 'employee'}`



CREATE TABLE employee (
	id INTEGER NOT NULL, 
	first_name VARCHAR(50), 
	last_name VARCHAR(50), 
	email VARCHAR(50), 
	gender VARCHAR(50), 
	role VARCHAR(50), 
	profile_pic TEXT, 
	working_hours INTEGER, 
	company VARCHAR(100), 
	base_salary INTEGER, 
	progress INTEGER, 
	CONSTRAINT employee_pkey PRIMARY KEY (id)
)

/*
3 rows from employee table:
id	first_name	last_name	email	gender	role	profile_pic	working_hours	company	base_salary	progress
26	Nanda	Purnama	caustickp@ucoz.ru	Male	internship	11	40	Sakura SSS	5313000	60
30	Wahyu	Sari	bwisont@oracle.com	Female	business development	19	45	Sakura Internal	6464000	76
37	Indra	Wijaya	cmcginley10@xrea.com	Male	operational	5	45	Sakura Krakatau Steel	7510000	96
*/
Invoking: `sql_db_query` with `{'query': 'SELECT company, COUNT(*) AS total_employees FROM employee GROUP B

{'input': 'List the total employees per company. Which company has most employees?',
 'output': 'The total number of employees per company are as follows:\n- Sakura SSS: 21 employees\n- Sakura Internal: 18 employees\n- Sakura Krakatau Steel: 11 employees\n\nThe company with the most employees is Sakura SSS with 21 employees.'}

In [4]:
def add_or_update_employee(employee_data):
    prompt = f"""
    Give me only the SQL query (no additional information or explanation):
    Given the following table schema: List of employees in the app, contains first_name, last_name, email, gender, role, and working_hours (weekly).
    Write an SQL query to insert or update the employee data based on this information: {employee_data}
    """
    
    # Generate SQL query
    response = agent_executor.invoke(prompt)
    sql_query = response

    # # Extract only the SQL part if it includes any additional text
    # if "```sql" in sql_query and "```" in sql_query:
    #     sql_query = sql_query.split("```sql")[1].split("```")[0].strip()

    print(sql_query)
    
    # try:
    #     session.execute(sql_query)
    #     session.commit()
    #     print("Employee data added/updated successfully.")
    # except Exception as e:
    #     session.rollback()
    #     print("Error:", str(e))

# Example employee data to be added or updated
employee_data = {
    "first_name": "John",
    "last_name": "Doe",
    "email": "john.doe@example.com",
    "gender": "Male",
    "role": "Engineer",
    "working_hours": 40
}

add_or_update_employee(employee_data)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


absensi_harian, employee
Invoking: `sql_db_schema` with `{'table_names': 'employee'}`



CREATE TABLE employee (
	id INTEGER NOT NULL, 
	first_name VARCHAR(50), 
	last_name VARCHAR(50), 
	email VARCHAR(50), 
	gender VARCHAR(50), 
	role VARCHAR(50), 
	profile_pic TEXT, 
	working_hours INTEGER, 
	company VARCHAR(100), 
	base_salary INTEGER, 
	progress INTEGER, 
	CONSTRAINT employee_pkey PRIMARY KEY (id)
)

/*
3 rows from employee table:
id	first_name	last_name	email	gender	role	profile_pic	working_hours	company	base_salary	progress
26	Nanda	Purnama	caustickp@ucoz.ru	Male	internship	11	40	Sakura SSS	5313000	60
30	Wahyu	Sari	bwisont@oracle.com	Female	business development	19	45	Sakura Internal	6464000	76
37	Indra	Wijaya	cmcginley10@xrea.com	Male	operational	5	45	Sakura Krakatau Steel	7510000	96
*/I don't know.

> Finished chain.
{'input': "\n    Give me only the SQL query (no additional information or e

In [5]:
import json 
examples = [
    {
        "input": "Add new employee Claire Brown, brown@example.com, Female, role HR, working hours 35, company Sakura SSS salary 60000 progress 40.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, gender, role, working_hours, company, base_salary, progress)
        VALUES ('Claire', 'Brown', 'claire.brown@example.com', 'Female', 'HR', 35, 'Sakura SSS', 60000, 40)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            gender = EXCLUDED.gender,
            role = EXCLUDED.role,
            working_hours = EXCLUDED.working_hours,
            company = EXCLUDED.company,
            base_salary = EXCLUDED.base_salary,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Update employee with email john.doe@example.com to role Senior Engineer and base salary 90000.",
        "query": """
        UPDATE employee
        SET role = 'Senior Engineer', base_salary = 90000
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Insert employee Bob Johnson email bob.johnson@example.com role Manager.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role)
        VALUES ('Bob', 'Johnson', 'bob.johnson@example.com', 'Manager')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Add new employee Alice, email alice.smith@example.com, working hours 40.",
        "query": """
        INSERT INTO employee (first_name, email, working_hours)
        VALUES ('Alice', 'alice.smith@example.com', 40)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            working_hours = EXCLUDED.working_hours;
        """
    },
    {
        "input": "Insert a new employee named Tom Brown with email tom.brown@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Tom', 'Brown', 'tom.brown@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Add new employee, email sarah.connor@example.com, gender Female.",
        "query": """
        INSERT INTO employee (email, gender)
        VALUES ('sarah.connor@example.com', 'Female')
        ON CONFLICT (email)
        DO UPDATE SET 
            gender = EXCLUDED.gender;
        """
    },
    {
        "input": "Update employee with email linda.white@example.com, working 45 hours weekly and company Tech Solutions.",
        "query": """
        UPDATE employee
        SET working_hours = 45, company = 'Tech Solutions'
        WHERE email = 'linda.white@example.com';
        """
    },
    {
        "input": "Add new employee Mike Davis, email mike.davis@example.com, company Business Inc.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, company)
        VALUES ('Mike', 'Davis', 'mike.davis@example.com', 'Business Inc')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            company = EXCLUDED.company;
        """
    },
    {
        "input": "Update employee john.doe@example.com to be gender Male and progress 90.",
        "query": """
        UPDATE employee
        SET gender = 'Male', progress = 90
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Insert employee Jane Doe with email jane.doe@example.com and role Engineer.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role)
        VALUES ('Jane', 'Doe', 'jane.doe@example.com', 'Engineer')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Add employee Emma Stone, email emma.stone@example.com, role HR and progress 75.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role, progress)
        VALUES ('Emma', 'Stone', 'emma.stone@example.com', 'HR', 75)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Insert a new employee named Jack Black, email jack.black@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Jack', 'Black', 'jack.black@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Add new employee Sara Connor, email sara.connor@example.com, base_salary 85000.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, base_salary)
        VALUES ('Sara', 'Connor', 'sara.connor@example.com', 85000)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Update employee with email chris.evans@example.com, role Developer and working_hours 40.",
        "query": """
        UPDATE employee
        SET role = 'Developer', working_hours = 40
        WHERE email = 'chris.evans@example.com';
        """
    },
    {
        "input": "Add new employee Anna Taylor, email anna.taylor@example.com, gender Female.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, gender)
        VALUES ('Anna', 'Taylor', 'anna.taylor@example.com', 'Female')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            gender = EXCLUDED.gender;
        """
    },
    {
        "input": "Insert employee Paul Walker with email paul.walker@example.com, progress 80.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, progress)
        VALUES ('Paul', 'Walker', 'paul.walker@example.com', 80)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Update employee Paul, email paul@example.com, role Manager.",
        "query": """
        UPDATE employee
        SET role = 'Manager'
        WHERE email = 'paul@example.com';
        """
    },
    {
        "input": "Update those whose company is Sakura SSS to add progress by 2.",
        "query": """
        UPDATE employee
        SET progress = progress + 2
        WHERE company = 'Sakura SSS';
        """
    },
    {
        "input": "Increase salary by 1 million for employee with email john.doe@example.com.",
        "query": """
        UPDATE employee
        SET base_salary = base_salary + 1000000
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Add progress by 5 for employees in Tech Solutions.",
        "query": """
        UPDATE employee
        SET progress = progress + 5
        WHERE company = 'Tech Solutions';
        """
    },
    {
        "input": "Set working_hours to 40 for employees with role Developer.",
        "query": """
        UPDATE employee
        SET working_hours = 40
        WHERE role = 'Developer';
        """
    },
    {
        "input": "Update base_salary to 75000 for employees in company Business Inc.",
        "query": """
        UPDATE employee
        SET base_salary = 75000
        WHERE company = 'Business Inc';
        """
    },
    {
        "input": "Increase progress by 3 for employees with email ending in '@example.com'.",
        "query": """
        UPDATE employee
        SET progress = progress + 3
        WHERE email LIKE '%@example.com';
        """
    },
    {
        "input": "Add employee Alice with email alice@example.com, role Admin, gender Female.",
        "query": """
        INSERT INTO employee (first_name, email, role, gender)
        VALUES ('Alice', 'alice@example.com', 'Admin', 'Female')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role,
            gender = EXCLUDED.gender;
        """
    },
    {
        "input": "Update working hours to 35 for employees with gender Female.",
        "query": """
        UPDATE employee
        SET working_hours = 35
        WHERE gender = 'Female';
        """
    },
    {
        "input": "Set salary to 65000 and progress to 80 for John Doe.",
        "query": """
        UPDATE employee
        SET base_salary = 65000, progress = 80
        WHERE first_name = 'John' AND last_name = 'Doe';
        """
    },
    {
        "input": "Add new employee without providing role: name Tom Hanks, email tom.hanks@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Tom', 'Hanks', 'tom.hanks@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Update company to 'Sakura Internal' for employees with base_salary greater than 70000.",
        "query": """
        UPDATE employee
        SET company = 'New Corp'
        WHERE base_salary > 70000;
        """
    },
    {
        "input": "Increase working hours by 2 for employees with progress less than 50.",
        "query": """
        UPDATE employee
        SET working_hours = working_hours + 2
        WHERE progress < 50;
        """
    }, 
    {
        "input": "Add employee Michael, email michael@example.com, role Designer, base salary 55000.",
        "query": """
        INSERT INTO employee (first_name, email, role, base_salary)
        VALUES ('Michael', 'michael@example.com', 'Designer', 55000)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Update employee Sarah's working hours to 50, email sarah@example.com.",
        "query": """
        UPDATE employee
        SET working_hours = 50
        WHERE email = 'sarah@example.com';
        """
    },
    {
        "input": "Increase base salary for employees with progress under 60 by 5000.",
        "query": """
        UPDATE employee
        SET base_salary = base_salary + 5000
        WHERE progress < 60;
        """
    },
    {
        "input": "Add new hire with email tom.jones@example.com, first name Tom, last name Jones, role Support.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role)
        VALUES ('Tom', 'Jones', 'tom.jones@example.com', 'Support')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Update all employees in company XYZ to have progress 100.",
        "query": """
        UPDATE employee
        SET progress = 100
        WHERE company = 'XYZ';
        """
    },
    {
        "input": "Set salary to 90000 and progress to 70 for Jane with email jane.doe@example.com.",
        "query": """
        UPDATE employee
        SET base_salary = 90000, progress = 70
        WHERE email = 'jane.doe@example.com';
        """
    },
    {
        "input": "Add new employee David, email david.smith@example.com, working hours 30.",
        "query": """
        INSERT INTO employee (first_name, email, working_hours)
        VALUES ('David', 'david.smith@example.com', 30)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            working_hours = EXCLUDED.working_hours;
        """
    },
    {
        "input": "Update all Female employees to have a base salary of 60000.",
        "query": """
        UPDATE employee
        SET base_salary = 60000
        WHERE gender = 'Female';
        """
    },
    {
        "input": "Add employee John with email johnny@example.com, role Tester.",
        "query": """
        INSERT INTO employee (first_name, email, role)
        VALUES ('John', 'johnny@example.com', 'Tester')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Update employees whose progress is less than 40 to have role Intern.",
        "query": """
        UPDATE employee
        SET role = 'intern'
        WHERE progress < 40;
        """
    }, 
      {
        "input": "Tambah karyawan baru Claire Brown, brown@example.com, Perempuan, jabatan HR, jam kerja 35, perusahaan Sakura SSS gaji 60000 progres 40.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, gender, role, working_hours, company, base_salary, progress)
        VALUES ('Claire', 'Brown', 'claire.brown@example.com', 'Female', 'HR', 35, 'Sakura SSS', 60000, 40)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            gender = EXCLUDED.gender,
            role = EXCLUDED.role,
            working_hours = EXCLUDED.working_hours,
            company = EXCLUDED.company,
            base_salary = EXCLUDED.base_salary,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Perbarui karyawan dengan email john.doe@example.com ke jabatan Senior Engineer dan gaji pokok 90000.",
        "query": """
        UPDATE employee
        SET role = 'Senior Engineer', base_salary = 90000
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Masukkan karyawan Bob Johnson email bob.johnson@example.com jabatan Manajer.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role)
        VALUES ('Bob', 'Johnson', 'bob.johnson@example.com', 'Manager')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Tambah karyawan baru Alice, email alice.smith@example.com, jam kerja 40.",
        "query": """
        INSERT INTO employee (first_name, email, working_hours)
        VALUES ('Alice', 'alice.smith@example.com', 40)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            working_hours = EXCLUDED.working_hours;
        """
    },
    {
        "input": "Masukkan karyawan baru bernama Tom Brown dengan email tom.brown@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Tom', 'Brown', 'tom.brown@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Tambah karyawan baru, email sarah.connor@example.com, jenis kelamin Perempuan.",
        "query": """
        INSERT INTO employee (email, gender)
        VALUES ('sarah.connor@example.com', 'Female')
        ON CONFLICT (email)
        DO UPDATE SET 
            gender = EXCLUDED.gender;
        """
    },
    {
        "input": "Perbarui karyawan dengan email linda.white@example.com, bekerja 45 jam per minggu dan perusahaan Tech Solutions.",
        "query": """
        UPDATE employee
        SET working_hours = 45, company = 'Tech Solutions'
        WHERE email = 'linda.white@example.com';
        """
    },
    {
        "input": "Tambah karyawan baru Mike Davis, email mike.davis@example.com, perusahaan Business Inc.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, company)
        VALUES ('Mike', 'Davis', 'mike.davis@example.com', 'Business Inc')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            company = EXCLUDED.company;
        """
    },
    {
        "input": "Perbarui karyawan john.doe@example.com menjadi jenis kelamin Laki-laki dan progres 90.",
        "query": """
        UPDATE employee
        SET gender = 'Male', progress = 90
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Masukkan karyawan Jane Doe dengan email jane.doe@example.com dan jabatan Engineer.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role)
        VALUES ('Jane', 'Doe', 'jane.doe@example.com', 'Engineer')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Tambah karyawan Emma Stone, email emma.stone@example.com, jabatan HR dan progres 75.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role, progress)
        VALUES ('Emma', 'Stone', 'emma.stone@example.com', 'HR', 75)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Masukkan karyawan baru bernama Jack Black, email jack.black@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Jack', 'Black', 'jack.black@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Tambah karyawan baru Sara Connor, email sara.connor@example.com, gaji pokok 85000.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, base_salary)
        VALUES ('Sara', 'Connor', 'sara.connor@example.com', 85000)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Perbarui karyawan dengan email chris.evans@example.com, jabatan Developer dan jam kerja 40.",
        "query": """
        UPDATE employee
        SET role = 'Developer', working_hours = 40
        WHERE email = 'chris.evans@example.com';
        """
    },
    {
        "input": "Tambah karyawan baru Anna Taylor, email anna.taylor@example.com, jenis kelamin Perempuan.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, gender)
        VALUES ('Anna', 'Taylor', 'anna.taylor@example.com', 'Female')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            gender = EXCLUDED.gender;
        """
    },
    {
        "input": "Masukkan karyawan Paul Walker dengan email paul.walker@example.com, progres 80.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, progress)
        VALUES ('Paul', 'Walker', 'paul.walker@example.com', 80)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Perbarui karyawan Paul, email paul@example.com, jabatan Manajer.",
        "query": """
        UPDATE employee
        SET role = 'Manager'
        WHERE email = 'paul@example.com';
        """
    },
    {
        "input": "Perbarui mereka yang perusahaannya adalah Sakura SSS untuk menambah progres sebesar 2.",
        "query": """
        UPDATE employee
        SET progress = progress + 2
        WHERE company = 'Sakura SSS';
        """
    },
    {
        "input": "Tingkatkan gaji sebesar 1 juta untuk karyawan dengan email john.doe@example.com.",
        "query": """
        UPDATE employee
        SET base_salary = base_salary + 1000000
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Tambah progres sebesar 5 untuk karyawan di Tech Solutions.",
        "query": """
        UPDATE employee
        SET progress = progress + 5
        WHERE company = 'Tech Solutions';
        """
    },
    {
        "input": "Atur jam kerja menjadi 40 untuk karyawan dengan jabatan Developer.",
        "query": """
        UPDATE employee
        SET working_hours = 40
        WHERE role = 'Developer';
        """
    },
    {
        "input": "Perbarui gaji pokok menjadi 75000 untuk karyawan di perusahaan Business Inc.",
        "query": """
        UPDATE employee
        SET base_salary = 75000
        WHERE company = 'Business Inc';
        """
    },
    {
        "input": "Tingkatkan progres sebesar 3 untuk karyawan dengan email yang berakhir dengan '@example.com'.",
        "query": """
        UPDATE employee
        SET progress = progress + 3
        WHERE email LIKE '%@example.com';
        """
    },
    {
        "input": "Tambah karyawan Alice dengan email alice@example.com, jabatan Admin, jenis kelamin Perempuan.",
        "query": """
        INSERT INTO employee (first_name, email, role, gender)
        VALUES ('Alice', 'alice@example.com', 'Admin', 'Female')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role,
            gender = EXCLUDED.gender;
        """
    },
    {
        "input": "Perbarui jam kerja menjadi 35 untuk karyawan dengan jenis kelamin Perempuan.",
        "query": """
        UPDATE employee
        SET working_hours = 35
        WHERE gender = 'Female';
        """
    },
    {
        "input": "Atur gaji menjadi 65000 dan progres menjadi 80 untuk John Doe.",
        "query": """
        UPDATE employee
        SET base_salary = 65000, progress = 80
        WHERE first_name = 'John' AND last_name = 'Doe';
        """
    },
    {
        "input": "Tambah karyawan baru tanpa memberikan jabatan: nama Tom Hanks, email tom.hanks@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Tom', 'Hanks', 'tom.hanks@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Perbarui perusahaan menjadi 'Sakura Internal' untuk karyawan dengan gaji pokok lebih dari 70000.",
        "query": """
        UPDATE employee
        SET company = 'New Corp'
        WHERE base_salary > 70000;
        """
    },
    {
        "input": "Tingkatkan jam kerja sebesar 2 untuk karyawan dengan progres kurang dari 50.",
        "query": """
        UPDATE employee
        SET working_hours = working_hours + 2
        WHERE progress < 50;
        """
    },
    {
        "input": "Tambah karyawan Michael, email michael@example.com, jabatan Designer, gaji pokok 55000.",
        "query": """
        INSERT INTO employee (first_name, email, role, base_salary)
        VALUES ('Michael', 'michael@example.com', 'Designer', 55000)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Perbarui jam kerja karyawan Sarah menjadi 50, email sarah@example.com.",
        "query": """
        UPDATE employee
        SET working_hours = 50
        WHERE email = 'sarah@example.com';
        """
    },
    {
        "input": "Tingkatkan gaji pokok untuk karyawan dengan progres di bawah 60 sebesar 5000.",
        "query": """
        UPDATE employee
        SET base_salary = base_salary + 5000
        WHERE progress < 60;
        """
    },
    {
        "input": "Tambah karyawan baru dengan email tom.jones@example.com, nama depan Tom, nama belakang Jones, jabatan Support.",
        "query": """
        INSERT INTO employee (first_name, last_name, email, role)
        VALUES ('Tom', 'Jones', 'tom.jones@example.com', 'Support')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Perbarui semua karyawan di perusahaan XYZ untuk memiliki progres 100.",
        "query": """
        UPDATE employee
        SET progress = 100
        WHERE company = 'XYZ';
        """
    },
    {
        "input": "Atur gaji menjadi 90000 dan progres menjadi 70 untuk Jane dengan email jane.doe@example.com.",
        "query": """
        UPDATE employee
        SET base_salary = 90000, progress = 70
        WHERE email = 'jane.doe@example.com';
        """
    },
    {
        "input": "Tambah karyawan baru David, email david.smith@example.com, jam kerja 30.",
        "query": """
        INSERT INTO employee (first_name, email, working_hours)
        VALUES ('David', 'david.smith@example.com', 30)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            working_hours = EXCLUDED.working_hours;
        """
    },
    {
        "input": "Perbarui semua karyawan Perempuan untuk memiliki gaji pokok 60000.",
        "query": """
        UPDATE employee
        SET base_salary = 60000
        WHERE gender = 'Female';
        """
    },
    {
        "input": "Tambah karyawan John dengan email johnny@example.com, jabatan Tester.",
        "query": """
        INSERT INTO employee (first_name, email, role)
        VALUES ('John', 'johnny@example.com', 'Tester')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Perbarui karyawan yang progresnya kurang dari 40 untuk memiliki jabatan Intern.",
        "query": """
        UPDATE employee
        SET role = 'intern'
        WHERE progress < 40;
        """
    },
     {
        "input": "Update employee dengan email anita@example.com ke gaji baru 70000.",
        "query": """
        UPDATE employee
        SET base_salary = 70000
        WHERE email = 'anita@example.com';
        """
    },
    {
        "input": "Perbarui karyawan itu, email: rudi@example.com, progres 85.",
        "query": """
        UPDATE employee
        SET progress = 85
        WHERE email = 'rudi@example.com';
        """
    },
    {
        "input": "Insert karyawan baru dengan nama Sofia, email: sofia@example.com, peran Developer.",
        "query": """
        INSERT INTO employee (first_name, email, role)
        VALUES ('Sofia', 'sofia@example.com', 'Developer')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Perbarui karyawan bernama Ali dengan gaji 75000.",
        "query": """
        UPDATE employee
        SET base_salary = 75000
        WHERE first_name = 'Ali';
        """
    },
    {
        "input": "Tolong tambahkan karyawan baru, email: tom@example.com, dan jabatan Admin.",
        "query": """
        INSERT INTO employee (email, role)
        VALUES ('tom@example.com', 'Admin')
        ON CONFLICT (email)
        DO UPDATE SET 
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Update karyawan john.doe@example.com menjadi 40 jam kerja.",
        "query": """
        UPDATE employee
        SET working_hours = 40
        WHERE email = 'john.doe@example.com';
        """
    },
    {
        "input": "Tambahkan karyawan baru, nama lengkap Lisa, email: lisa@example.com.",
        "query": """
        INSERT INTO employee (first_name, last_name, email)
        VALUES ('Lisa', 'Smith', 'lisa@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            last_name = EXCLUDED.last_name;
        """
    },
    {
        "input": "Perbarui progres karyawan dengan email: diana@example.com menjadi 90.",
        "query": """
        UPDATE employee
        SET progress = 90
        WHERE email = 'diana@example.com';
        """
    },
    {
        "input": "Update employee email: adam@example.com ke jam kerja 36.",
        "query": """
        UPDATE employee
        SET working_hours = 36
        WHERE email = 'adam@example.com';
        """
    },
    {
        "input": "Insert karyawan baru: Nick, email: nick@example.com, gaji 50000.",
        "query": """
        INSERT INTO employee (first_name, email, base_salary)
        VALUES ('Nick', 'nick@example.com', 50000)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Perbarui, employee dengan email: jessica@example.com, perusahaan: Global Corp.",
        "query": """
        UPDATE employee
        SET company = 'Global Corp'
        WHERE email = 'jessica@example.com';
        """
    },
    {
        "input": "Tolong update karyawan, email: steve@example.com, role menjadi Manager.",
        "query": """
        UPDATE employee
        SET role = 'Manager'
        WHERE email = 'steve@example.com';
        """
    },
    {
        "input": "Tambah karyawan baru, email: bella@example.com, jabatan HR, gaji 65000.",
        "query": """
        INSERT INTO employee (email, role, base_salary)
        VALUES ('bella@example.com', 'HR', 65000)
        ON CONFLICT (email)
        DO UPDATE SET 
            role = EXCLUDED.role,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Update karyawan itu dengan nama: Chris dan email: chris@example.com.",
        "query": """
        INSERT INTO employee (first_name, email)
        VALUES ('Chris', 'chris@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name;
        """
    },
    {
        "input": "Perbarui gaji untuk karyawan di email: jenny@example.com, menjadi 72000.",
        "query": """
        UPDATE employee
        SET base_salary = 72000
        WHERE email = 'jenny@example.com';
        """
    },
    {
        "input": "Insert karyawan baru dengan email: karl@example.com dan jabatan Sales.",
        "query": """
        INSERT INTO employee (email, role)
        VALUES ('karl@example.com', 'Sales')
        ON CONFLICT (email)
        DO UPDATE SET 
            role = EXCLUDED.role;
        """
    },
    {
        "input": "Perbarui karyawan email: fiona@example.com, jam kerja menjadi 45.",
        "query": """
        UPDATE employee
        SET working_hours = 45
        WHERE email = 'fiona@example.com';
        """
    },
    {
        "input": "Update employee email: lara@example.com ke progres 80.",
        "query": """
        UPDATE employee
        SET progress = 80
        WHERE email = 'lara@example.com';
        """
    },
    {
        "input": "Masukkan karyawan baru, namanya: Rick, email: rick@example.com.",
        "query": """
        INSERT INTO employee (first_name, email)
        VALUES ('Rick', 'rick@example.com')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name;
        """
    },
    {
        "input": "Perbarui karyawan, email: henry@example.com, menjadi jabatan Engineer.",
        "query": """
        UPDATE employee
        SET role = 'Engineer'
        WHERE email = 'henry@example.com';
        """
    },
    {
        "input": "Update karyawan dengan email: miranda@example.com, jam kerja menjadi 50.",
        "query": """
        UPDATE employee
        SET working_hours = 50
        WHERE email = 'miranda@example.com';
        """
    },
    {
        "input": "Tambah karyawan baru, email: emma@example.com, jabatan Admin, gaji 48000.",
        "query": """
        INSERT INTO employee (email, role, base_salary)
        VALUES ('emma@example.com', 'Admin', 48000)
        ON CONFLICT (email)
        DO UPDATE SET 
            role = EXCLUDED.role,
            base_salary = EXCLUDED.base_salary;
        """
    },
    {
        "input": "Perbarui gaji karyawan, email: vincent@example.com, menjadi 66000.",
        "query": """
        UPDATE employee
        SET base_salary = 66000
        WHERE email = 'vincent@example.com';
        """
    },
    {
        "input": "Insert karyawan baru: Charles, email: charles@example.com, progres 75.",
        "query": """
        INSERT INTO employee (first_name, email, progress)
        VALUES ('Charles', 'charles@example.com', 75)
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            progress = EXCLUDED.progress;
        """
    },
    {
        "input": "Update employee dengan email: kathy@example.com, jabatan menjadi Support.",
        "query": """
        UPDATE employee
        SET role = 'Support'
        WHERE email = 'kathy@example.com';
        """
    },
    {
        "input": "Perbarui semua karyawan yang gajinya di atas 70000 ke progres 100.",
        "query": """
        UPDATE employee
        SET progress = 100
        WHERE base_salary > 70000;
        """
    },
    {
        "input": "Tolong masukkan karyawan baru: randy, email: randy@example.com, jabatan Manager.",
        "query": """
        INSERT INTO employee (first_name, email, role)
        VALUES ('Randy', 'randy@example.com', 'Manager')
        ON CONFLICT (email)
        DO UPDATE SET 
            first_name = EXCLUDED.first_name,
            role = EXCLUDED.role;
        """
    }
]


In [6]:
jsonl_content = []
system_content = "You are an assistant to outputs SQL queries for INSERT or UPDATE statements only."

for example in examples:
    message = {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": example["input"]},
            {"role": "assistant", "content": example["query"].strip()}
        ]
    }
    jsonl_content.append(json.dumps(message))

# Save to a JSONL file
with open('fine_tune_data.jsonl', 'w') as f:
    for line in jsonl_content:
        f.write(line + '\n')

print("Data has been written to fine_tune_data.jsonl")

Data has been written to fine_tune_data.jsonl


In [7]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

KeyboardInterrupt: 

In [1]:
import json

def stringify_assistant_content(filename, output_filename):
    with open(filename, 'r', encoding='utf-8') as infile, open(output_filename, 'w', encoding='utf-8') as outfile:
        for line in infile:
            data = json.loads(line)
            for message in data['messages']:
                if message['role'] == 'assistant' and isinstance(message['content'], dict):
                    message['content'] = json.dumps(message['content'], ensure_ascii=False)
            outfile.write(json.dumps(data, ensure_ascii=False) + '\n')

# Usage
input_filename = 'fine_tune_evaluation.jsonl'  # Replace with the path to your input JSONL file
output_filename = 'fine_tune_evaluation_output.jsonl'  # Replace with the path to your desired output JSONL file
stringify_assistant_content(input_filename, output_filename)
